# Visualize

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [10]:
#start_frame부터 80 frame의 영상을 clip함
def clip_video(video_path, start_frame, save_path):
    print("cliiping start")

    file_name = os.path.basename(video_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None

    # Set the starting frame
    print(file_name)
    output_path = save_path + '/' + file_name
    print(output_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), 
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Read and write frames
    frame_count = 0
    while frame_count < 1200:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        frame_count += 1

    # Release everything if job is finished
    cap.release()
    out.release()
    print("cliiping end")

    return output_path



In [10]:
video_path = '/home/sm32289/Cat_Pose/KeypointDetection_Yolo/Test_video/normal_cat_full/길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4'
start_frame = 100
save_path = '/home/sm32289/Cat_Pose/PipeLine(Result)/clip'

clip_video(video_path, start_frame, save_path)

cliiping start
길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4
/home/sm32289/Cat_Pose/PipeLine(Result)clip/길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4
cliiping end


'/home/sm32289/Cat_Pose/PipeLine(Result)/\x08clip/길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4'

In [11]:
def draw_keypoints(frame, keyframe_data, frame_width, frame_height):
    print("drawing start")
    if len(keyframe_data) == 30:
        for i in range(0,15):
            # Scale the normalized coordinates to the frame size
            x = int(keyframe_data[2 * i] * frame_width)
            y = int(keyframe_data[2 * i + 1] * frame_height)
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            
    print("drawing end")
    return frame


In [5]:
# Read the video
cap = cv2.VideoCapture('/home/sm32289/Cat_Pose/PipeLine(Result)/clip/강아지와 고양이가 쉽게 친해질 수 없는 이유ㅋㅋㅋㅋㅋㅋㅋㅋ.mp4')

# Get frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), 
    #                       (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/home/sm32289/Cat_Pose/PipeLine(Result)/clip_with_keypoints/output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))

frame_num = 100
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Load the corresponding .npy file
    keyframe_data = np.load(f'/home/sm32289/Cat_Pose/KeypointDetection_Yolo/predict2/normal_cat_full/keypoints/강아지와 고양이가 쉽게 친해질 수 없는 이유ㅋㅋㅋㅋㅋㅋㅋㅋ/{frame_num}.npy')

    # Overlay the keyframe data onto the frame
    frame = draw_keypoints(frame, keyframe_data, frame_width, frame_height)

    # Write the frame to the output video
    out.write(frame)

    frame_count += 1
    frame_num += 1
    if frame_count >= 80:  # Assuming there are exactly 80 frames and keyframes
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: '/home/sm32289/Cat_Pose/KeypointDetection(Yolo)/predict/Normal/keypoints/강아지와 고양이가 쉽게 친해질 수 없는 이유ㅋㅋㅋㅋㅋㅋㅋㅋ/100.npy'

In [12]:
def get_video_with_keypoints(video_path, keypoint_path, frame_number, output_clip_path, output_clip_keypoint_path):
    
    output_path = clip_video(video_path, frame_number, output_clip_path)
    path_list = list(output_path.split('/'))
    video_name = path_list[-1]
    output_clip_path = output_clip_path + '/' + video_name
    output_clip_keypoint_path = output_clip_keypoint_path + '/' + video_name

    # Read the video
    cap = cv2.VideoCapture(output_path)
    
    # Get frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_clip_keypoint_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))

    frame_num = frame_number
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Load the corresponding .npy file
        keyframe_data = np.load(f'{keypoint_path}/{frame_num}.npy')

        # Overlay the keyframe data onto the frame
        frame = draw_keypoints(frame, keyframe_data, frame_width, frame_height)

        # Write the frame to the output video
        out.write(frame)
        
        frame_count += 1
        frame_num += 1
        if frame_count >= 1200:  # Assuming there are exactly 80 frames and keyframes
            break
    # Release resources
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

    

In [13]:
video_path = '/home/sm32289/Cat_Pose/KeypointDetection_Yolo/Test_video/normal_cat_full/길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4'
keypoint_path = '/home/sm32289/Cat_Pose/KeypointDetection_Yolo/predict/Normal/keypoints/길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의)'
frame_number = 3600
output_clip_path = '/home/sm32289/Cat_Pose/PipeLine(Result)/clip'
output_clip_keypoint_path = '/home/sm32289/Cat_Pose/PipeLine(Result)/clip_with_keypoints'

get_video_with_keypoints(video_path, keypoint_path, frame_number, output_clip_path, output_clip_keypoint_path)

cliiping start
길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4
/home/sm32289/Cat_Pose/PipeLine(Result)clip/길고양이 데려다가 확대하고 안락삶 시켜버렸습니다(난독주의).mp4
cliiping end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing end
drawing start
drawing 